# Route Plan Analysis

Use this notebook to map and analyze a routing plan from Azavea's [School Bus Routing Optimization tool](https://github.com/azavea/bus-plan). Update the cell below to point to local copies of solver output.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline  

import map_solver 
import student_ride_times

In [ ]:
routes = "< Path to router output >"
stop_assignment = "< Path to solver student-stop assignment csv >"

### Proposed bus plan

In [ ]:
v = map_solver.visualize_plan(routes)

In [ ]:
v['map']

In [ ]:
v['animation']

### Route stats

In [ ]:
# calculate student ride time
ride_times = student_ride_times.get_student_ride_times(routes, stop_assignment)
# load routing output
plan = map_solver.get_csv(routes)

garages = plan[plan['origin_type'] == 'garage']
schools = plan[plan['destination_type'] == 'school']
stops = plan[plan['origin_type'] == 'stop']

In [ ]:
'Total buses: ' + str(plan['route_id'].nunique())

In [ ]:
'Total bus stops: ' + str(stops.groupby(['route_id', 'origin_id']).ngroups)

In [ ]:
'Total unique bus stops: ' + str(stops['origin_id'].nunique())

In [ ]:
'Garages used: ' + str(garages['origin_id'].nunique())

In [ ]:
with open(stop_assignment) as f:
    s = sum([len(line.split(',')) - 2 for line in f])
'Total students: ' + str(s)

In [ ]:
'Students not routed: ' + str(1056 - s)

### Student ride time

In [ ]:
import numpy as np
times = round(pd.to_numeric(ride_times['duration']) / 60, 2)
times.describe()

In [ ]:
sns.set_style(('whitegrid'))
plt.figure(figsize=(14,7))
dens = sns.kdeplot(times, shade=True, color="b")
dens.set(xlabel='Ride time (minutes)')
dens.set_title('Proposed distribution of student ride times')

### Bus usage

In [ ]:
def get_route_time(plan, route):
    time = plan[plan['route_id'] == route]['time']
    return (time.max() - time.min()) / 60

In [ ]:
route_times = [get_route_time(plan, r) for r in plan['route_id'].unique() ]

In [ ]:
'Total active bus time (hours): ' + str(sum(route_times) / 60)

In [ ]:
'Average total route time (minutes): ' + str(np.mean(route_times))